In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Saúde', usecols=['Cod Municipio','MSAU01','MSAU03','MSAU04','MSAU05','MSAU06'])
df18

,Cod Municipio,MSAU01,MSAU03,MSAU04,MSAU05,MSAU06
0,1100015,Secretaria exclusiva,Masculino,46,Preta,Ensino médio (2º Grau) completo
1,1100023,Secretaria exclusiva,Masculino,37,Branca,Especialização
2,1100031,Secretaria exclusiva,Masculino,45,Parda,Ensino superior completo
3,1100049,Secretaria exclusiva,Feminino,45,Branca,Especialização
4,1100056,Secretaria exclusiva,Feminino,62,Branca,Ensino médio (2º Grau) completo
...,...,...,...,...,...,...
5565,5222005,Secretaria exclusiva,Feminino,31,Parda,Especialização
5566,5222054,Secretaria exclusiva,Masculino,58,Branca,Ensino médio (2º Grau) completo
5567,5222203,Secretaria exclusiva,Masculino,37,Parda,Ensino superior completo
5568,5222302,Secretaria exclusiva,Feminino,32,Branca,Ensino superior incompleto


In [4]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Saúde', usecols=['Mun','Msau01','Msau03','Msau04','Msau05','Msau06'])
df21

,Mun,Msau01,Msau03,Msau04,Msau05,Msau06
0,Alta Floresta DOeste,Secretaria exclusiva,Masculino,37,Parda,Especialização
1,Ariquemes,Secretaria exclusiva,Feminino,44,Branca,Especialização
2,Cabixi,Secretaria exclusiva,Masculino,49,Parda,Ensino superior completo
3,Cacoal,Secretaria exclusiva,Feminino,42,Parda,Ensino superior completo
4,Cerejeiras,Secretaria exclusiva,Masculino,35,Branca,Ensino superior completo
...,...,...,...,...,...,...
5565,Vianópolis,Secretaria exclusiva,Feminino,32,Parda,Ensino superior completo
5566,Vicentinópolis,Secretaria exclusiva,Masculino,61,Parda,Ensino médio (2º Grau) completo
5567,Vila Boa,Secretaria exclusiva,Feminino,45,Preta,Ensino médio (2º Grau) incompleto
5568,Vila Propício,Secretaria exclusiva,Feminino,39,Branca,Ensino superior completo


In [5]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


In [6]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MSAU01':'caracterizacao_orgao_gestor',
                        'MSAU03':'genero',
                        'MSAU04' :'idade',
                        'MSAU05':'cor_raca',
                        'MSAU06':'grau_instrucao'}) 

In [7]:
df21= df21.rename(columns={'Mun':'id_municipio',
                        'Msau01':'caracterizacao_orgao_gestor',
                        'Msau03':'genero',
                        'Msau04' :'idade',
                        'Msau05':'cor_raca',
                        'Msau06':'grau_instrucao'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
df

,id_municipio,caracterizacao_orgao_gestor,genero,idade,cor_raca,grau_instrucao,ano
0,1100015,Secretaria exclusiva,Masculino,46,Preta,Ensino médio (2º Grau) completo,2018
1,1100023,Secretaria exclusiva,Masculino,37,Branca,Especialização,2018
2,1100031,Secretaria exclusiva,Masculino,45,Parda,Ensino superior completo,2018
3,1100049,Secretaria exclusiva,Feminino,45,Branca,Especialização,2018
4,1100056,Secretaria exclusiva,Feminino,62,Branca,Ensino médio (2º Grau) completo,2018
...,...,...,...,...,...,...,...
5565,Vianópolis,Secretaria exclusiva,Feminino,32,Parda,Ensino superior completo,2021
5566,Vicentinópolis,Secretaria exclusiva,Masculino,61,Parda,Ensino médio (2º Grau) completo,2021
5567,Vila Boa,Secretaria exclusiva,Feminino,45,Preta,Ensino médio (2º Grau) incompleto,2021
5568,Vila Propício,Secretaria exclusiva,Feminino,39,Branca,Ensino superior completo,2021


In [11]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [12]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [13]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [14]:
df = df.drop(['CodMun'], axis=1)

In [15]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'Mun':'nome_municipio'}) #padronizando as colunas

In [16]:
def transformar(nome):
    nome = re.sub(r"\bdiretamente\nà \b", "diretamente a ", nome)
    return nome

In [17]:
df['caracterizacao_orgao_gestor']= df['caracterizacao_orgao_gestor'].apply(transformar) ##padronizando os nomes com erro de escrita


In [18]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='(**) Sem gestor','Sem titular',df['genero'])

#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**) Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])


#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='(**) Sem gestor','Sem titular',df['cor_raca'])

#padronizando os dados da coluna 'escolaridade'
df['grau_instrucao']=np.where(df['grau_instrucao']=='(**) Sem gestor','Sem titular',df['grau_instrucao'])


In [20]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [21]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [22]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [23]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio','caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

In [26]:
# criando dicionário
dict_esco = {'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino fundamental (1º Grau) completo':'Até Ensino Fundamental',
             'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [27]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar

In [28]:
df['grau_instrucao'].unique()

array(['Até Ensino Médio', 'Até Pós Graduação ou Mestrado',
       'Até Ensino Superior Completo', 'Até Ensino Fundamental',
       'Até Doutorado', 'Recusa', 'Sem titular'], dtype=object)

Subindo para o GBQ

In [29]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          5570 non-null   int64   
 1   sigla_uf                     5570 non-null   object  
 2   nome_municipio               5570 non-null   object  
 3   id_municipio                 5570 non-null   object  
 4   caracterizacao_orgao_gestor  5570 non-null   object  
 5   genero                       5570 non-null   object  
 6   cor_raca                     5570 non-null   object  
 7   grau_instrucao               5570 non-null   object  
 8   faixa_etaria                 5518 non-null   category
dtypes: category(1), int64(1), object(7)
memory usage: 397.3+ KB


In [31]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [32]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_saude_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=b42ac789-4322-4ee0-a4a3-71783a3c118d>